# Creating a Vector Database for LangChain Prompt Engineering

- using the KPoEM dataset—after applying min–max scaling with a lower bound of 0.2—embedded with the latest KcELECTRA backbone model, as a LangChain vector store.

## 1. Basic setup and library imports
- Import Libraries and Set Configuration

In [1]:
!pip install -q datasets pandas faiss-cpu
!pip install -q langchain langchain-core langchain-community langchain-huggingface

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1

In [2]:
from langchain.vectorstores import FAISS
from datasets import load_dataset
import pandas as pd
from langchain.docstore.document import Document
import torch
from collections import Counter
from typing import List
from dataclasses import dataclass
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel
import numpy as np
from langchain.embeddings.base import Embeddings

## 2. Loading and Reprocessing the KPoEM Dataset by Row and by Poem
- Load the AKS-DHLAB KPoEM dataset from Hugging Face both at the line level and poem level, then merge them on their common columns to create a new DataFrame."
- For the combined KPoEM (lines + poems) dataset, apply min–max scaling with a scaling factor of 0.2 to the emotion annotations from all five annotators, and store the results in a new column.

In [ ]:
# KPoEM 데이터셋(작품 단위) 로드
df_poem_ = load_dataset(
    "csv",  # csv 포맷으로 읽기
    data_files={
        "train": "hf://datasets/AKS-DHLAB/KPoEM/KPoEM_poem_dataset_v4.tsv",  # Hugging Face에 저장된 TSV 파일 경로
    },
    delimiter="\t",  # TSV 파일이므로 탭 구분자 지정
    encoding="utf-8",  # UTF-8 인코딩 (필요 시 "utf-8-sig" 사용)
    quoting=3,        # csv.QUOTE_NONE → 인용부호 무시
)

# HuggingFace Dataset → Pandas DataFrame 변환
df_poem = df_poem_["train"].to_pandas()

# 데이터 확인 (상위 5행 미리보기)
df_poem.head()

,seg_id,poem_id,text,sub_title,title,poetry_book,poet,annotator_01,annotator_02,annotator_03,annotator_04,annotator_05
0,1,1,"죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를, 잎새에 이는 바람에도 나는 괴로...",None,서시,하늘과 바람과 별과 시,윤동주,"불안/걱정, 비장함, 서러움, 슬픔, 안타까움/실망, 부끄러움, 죄책감, 패배/자기혐오","패배/자기혐오, 서러움, 비장함, 부끄러움, 아껴주는, 슬픔, 불안/걱정, 죄책감","깨달음, 흐뭇함(귀여움/예쁨), 고마움, 힘듦/지침, 안타까움/실망","비장함, 뿌듯함, 슬픔","비장함, 감동/감탄, 깨달음, 서러움"
1,2,2,산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다. 우물 속에는 ...,None,자화상,하늘과 바람과 별과 시,윤동주,"불쌍함/연민, 아껴주는, 신기함/관심, 흐뭇함(귀여움/예쁨), 서러움, 짜증, 지긋...","신기함/관심, 증오/혐오, 불쌍함/연민, 화남/분노, 서러움, 슬픔, 아껴주는, 패...","안타까움/실망, 서러움, 슬픔, 깨달음, 흐뭇함(귀여움/예쁨)","불평/불만, 안타까움/실망, 의심/불신, 슬픔","불쌍함/연민, 불안/걱정"
2,3,3,쫓아오든 햇빛인데 지금 교회당 꼭대기 십자가에 걸리었습니다. 첨탑(尖塔)이 저렇게...,None,십자가,하늘과 바람과 별과 시,윤동주,"서러움, 패배/자기혐오, 존경, 감동/감탄, 비장함, 불쌍함/연민, 불안/걱정, 죄책감","감동/감탄, 놀람, 불쌍함/연민, 비장함, 슬픔, 행복, 아껴주는, 불안/걱정","절망, 깨달음, 서러움, 힘듦/지침, 존경, 비장함","당황/난처, 힘듦/지침, 놀람, 슬픔, 불안/걱정","깨달음, 비장함, 존경, 신기함/관심"
3,4,4,바람이 어디로부터 불어 와 어디로 불려 가는 것일까 바람이 부는데 내 괴로움에는 ...,None,바람이 불어,하늘과 바람과 별과 시,윤동주,"슬픔, 서러움, 한심함, 패배/자기혐오, 죄책감, 부끄러움","비장함, 서러움, 슬픔, 불안/걱정, 패배/자기혐오, 죄책감","힘듦/지침, 안타까움/실망, 서러움, 절망, 깨달음","힘듦/지침, 당황/난처, 불안/걱정, 불평/불만, 슬픔, 부끄러움, 패배/자기혐오","깨달음, 불쌍함/연민, 불안/걱정, 서러움, 패배/자기혐오, 힘듦/지침"
4,5,5,고향에 돌아온 날 밤에 내 백골(白骨)이 따라와 한방에 누웠다. 어둔 방은 우주로...,None,또 다른 고향,하늘과 바람과 별과 시,윤동주,"불쌍함/연민, 비장함, 공포/무서움, 슬픔, 서러움, 의심/불신, 깨달음, 기대감","힘듦/지침, 놀람, 서러움, 슬픔, 감동/감탄, 존경, 죄책감, 패배/자기혐오, 비...","절망, 깨달음, 서러움, 힘듦/지침, 안타까움/실망","슬픔, 힘듦/지침, 절망, 기대감, 안타까움/실망, 불안/걱정","공포/무서움, 당황/난처, 놀람, 불안/걱정, 비장함"


In [17]:
# KPoEM 데이터셋(행/시구 단위) 로드
df_line_ = load_dataset(
    "csv",  # CSV 포맷으로 읽기
    data_files={
        "train": "hf://datasets/AKS-DHLAB/KPoEM/KPoEM_line_dataset_v4.tsv",  # Hugging Face에 저장된 시구 단위 데이터 경로
    },
    delimiter="\t",  # 탭 구분자 지정
    encoding="utf-8",  # UTF-8 인코딩 (필요 시 "utf-8-sig" 사용 가능)
    quoting=3,        # csv.QUOTE_NONE → 인용부호 무시
)

# HuggingFace Dataset → Pandas DataFrame 변환
df_line = df_line_["train"].to_pandas()

# 데이터 확인 (상위 5행 미리보기)
df_line.head()

,line_id,poem_id,text,sub_title,title,poet,annotator_01,annotator_02,annotator_03,annotator_04,annotator_05
0,1,1,죽는 날까지 하늘을 우러러,None,서시,윤동주,비장함,비장함,"뿌듯함, 비장함","비장함, 뿌듯함, 감동/감탄","비장함, 서러움, 슬픔"
1,2,1,"한 점 부끄럼이 없기를,",None,서시,윤동주,"부끄러움, 비장함","부끄러움, 비장함, 기대감, 불안/걱정, 서러움, 슬픔","깨달음, 비장함, 뿌듯함","비장함, 부끄러움, 기대감",비장함
2,3,1,잎새에 이는 바람에도,None,서시,윤동주,"기대감, 신기함/관심","기대감, 불안/걱정, 비장함","슬픔, 서러움, 불안/걱정, 당황/난처","비장함, 슬픔","감동/감탄, 신기함/관심, 편안/쾌적, 기대감"
3,4,1,나는 괴로워했다.,None,서시,윤동주,"절망, 슬픔, 패배/자기혐오","절망, 슬픔, 패배/자기혐오, 죄책감, 힘듦/지침, 비장함","당황/난처, 서러움, 죄책감, 패배/자기혐오","비장함, 슬픔, 패배/자기혐오, 절망, 힘듦/지침","슬픔, 서러움, 절망, 힘듦/지침, 패배/자기혐오"
4,5,1,별을 노래하는 마음으로,None,서시,윤동주,"기쁨, 신기함/관심, 즐거움/신남, 흐뭇함(귀여움/예쁨), 뿌듯함","기쁨, 뿌듯함, 감동/감탄, 슬픔, 비장함, 아껴주는, 환영/호의, 기대감","고마움, 기대감, 기쁨, 아껴주는, 흐뭇함(귀여움/예쁨)","감동/감탄, 기대감, 기쁨, 아껴주는, 행복","즐거움/신남, 기대감, 기쁨, 행복"


In [6]:
# line 단위 DataFrame과 poem 단위 DataFrame의 공통 컬럼 찾기
common_cols = list(set(df_line.columns) & set(df_poem.columns))

# 공통 컬럼 기준으로 두 DataFrame을 연결(concatenate)하여 하나의 DataFrame 생성
df_merged = pd.concat([df_line[common_cols], df_poem[common_cols]], ignore_index=True)

# 원하는 컬럼 순서를 리스트로 정의
order = ["poem_id", "text", "sub_title", "title", "poet", 
         "annotator_01", "annotator_02", "annotator_03", 
         "annotator_04", "annotator_05"]

# order 리스트에 정의된 컬럼 중 실제 존재하는 컬럼만 선택
cols = [c for c in order if c in df_merged.columns]

# DataFrame의 컬럼 순서를 지정한 순서로 재정렬
df_merged = df_merged[cols]

In [18]:
# 함수 정의: 행(row) 단위 감정 주석 Min–Max 스케일링
def minmax_scale_annotations(row, annotator_cols, label_list):
    """
    단일 행(row)에 대해 감정 주석을 Min–Max 스케일링(0.0~1.0)하여 반환합니다.
    각 감정 값은 (해당 감정을 선택한 어노테이터 수 / 전체 어노테이터 수)로 계산됩니다.

    매개변수:
        row (pd.Series): 데이터프레임의 한 행
        annotator_cols (list): 어노테이터 감정 주석이 들어 있는 컬럼 이름 리스트
        label_list (list): 전체 감정 라벨 리스트 (44개 감정)

    반환값:
        dict: {감정라벨: 스케일링된 값} 형태의 딕셔너리
              (어노테이터가 5명인 경우 0, 0.2, 0.4, 0.6, 0.8, 1.0 값만 가능)
    """
    # 각 감정이 몇 번 선택되었는지 카운트하기 위한 Counter 생성
    counts = Counter()
    total = len(annotator_cols)  # 전체 어노테이터 수 (예: 5명)

    # 현재 행의 모든 어노테이터 컬럼을 순회
    for col in annotator_cols:
        ann = row[col]
        if pd.isna(ann):  # NaN이면(주석이 없으면) 건너뛰기
            continue

        # 하나의 셀에 여러 감정이 콤마로 구분되어 있을 경우 분리 후 공백 제거
        labels = [l.strip() for l in str(ann).split(",")]
        counts.update(labels)  # 각 감정의 등장 횟수 누적

    # Min–Max 스케일링 (선택 비율 계산)
    # 예: 슬픔이 3명 → 3 / 5 = 0.6
    scaled = {lab: counts.get(lab, 0) / total for lab in label_list}

    return scaled


In [19]:
# 행(row) 단위 감정 주석 Min–Max 스케일링 실행
# KOTE 44개 감정 라벨 정의
labels = [
    '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노',
    '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함',
    '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함',
    '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침',
    '즐거움/신남', '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처',
    '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정',
    '기쁨', '안심/신뢰'
]

# 5명의 어노테이터 감정 주석 컬럼 이름 지정
annotator_cols = ["annotator_01", "annotator_02", "annotator_03", "annotator_04", "annotator_05"]

# df_merged의 각 행에 대해 minmax_scale_annotations 함수 적용
# 결과: {감정: 스케일링 값} 형태의 dict가 들어있는 시리즈 생성
scaled_outputs = df_merged.apply(
    lambda row: minmax_scale_annotations(row, annotator_cols, labels),
    axis=1
)

# dict 시리즈를 리스트로 변환 후 DataFrame으로 확장
# 최종 결과: 각 감정 라벨이 컬럼으로, 스케일링된 값이 행별로 들어있는 감정 벡터 데이터프레임 생성
scaled_df = pd.DataFrame(list(scaled_outputs))
scaled_df


,불평/불만,환영/호의,감동/감탄,지긋지긋,고마움,슬픔,화남/분노,존경,기대감,우쭐댐/무시함,...,경악,부담/안_내킴,서러움,재미없음,불쌍함/연민,놀람,행복,불안/걱정,기쁨,안심/신뢰
0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.4,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0
2,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.0,0.6,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.4,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.2,0.4,0.0,0.2,0.2,0.0,0.0,0.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,0.4,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,...,0.4,0.2,0.2,0.0,0.2,0.0,0.0,0.6,0.0,0.0
7618,0.0,0.0,0.0,0.4,0.0,0.4,0.0,0.0,0.0,0.0,...,0.2,0.2,0.2,0.2,0.4,0.0,0.0,0.6,0.0,0.0
7619,0.0,0.0,0.0,0.4,0.0,0.2,0.2,0.0,0.0,0.0,...,0.6,0.0,0.0,0.0,0.2,0.0,0.0,0.6,0.0,0.0
7620,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.4,0.2,...,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.8,0.0,0.0


In [13]:
# 각 행에서 0.0보다 큰 값만 딕셔너리로 추출
dict_series = scaled_df.apply(lambda row: row[row > 0.0].to_dict(), axis=1)

# Series of dicts → list of dicts
dict_list = dict_series.tolist()

# 각 dict를 value 기준으로 내림차순 정렬
sorted_dict_list = [
    dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    for d in dict_list
]
sorted_dict_list[:2]

[{'비장함': 1.0, '뿌듯함': 0.4, '감동/감탄': 0.2, '슬픔': 0.2, '서러움': 0.2},
 {'비장함': 1.0,
  '부끄러움': 0.6,
  '기대감': 0.4,
  '슬픔': 0.2,
  '뿌듯함': 0.2,
  '깨달음': 0.2,
  '서러움': 0.2,
  '불안/걱정': 0.2}]

In [20]:
# dict_list를 'scaled_labels'라는 새 컬럼으로 추가
df_merged["scaled_labels"] = sorted_dict_list

In [22]:
df_sum = df_merged.reset_index(drop=True)
df_sum.insert(0, "id", df_sum.index)
df_sum

,id,poem_id,text,sub_title,title,poet,annotator_01,annotator_02,annotator_03,annotator_04,annotator_05,scaled_labels
0,0,1,죽는 날까지 하늘을 우러러,None,서시,윤동주,비장함,비장함,"뿌듯함, 비장함","비장함, 뿌듯함, 감동/감탄","비장함, 서러움, 슬픔","{'비장함': 1.0, '뿌듯함': 0.4, '감동/감탄': 0.2, '슬픔': 0..."
1,1,1,"한 점 부끄럼이 없기를,",None,서시,윤동주,"부끄러움, 비장함","부끄러움, 비장함, 기대감, 불안/걱정, 서러움, 슬픔","깨달음, 비장함, 뿌듯함","비장함, 부끄러움, 기대감",비장함,"{'비장함': 1.0, '부끄러움': 0.6, '기대감': 0.4, '슬픔': 0...."
2,2,1,잎새에 이는 바람에도,None,서시,윤동주,"기대감, 신기함/관심","기대감, 불안/걱정, 비장함","슬픔, 서러움, 불안/걱정, 당황/난처","비장함, 슬픔","감동/감탄, 신기함/관심, 편안/쾌적, 기대감","{'기대감': 0.6, '슬픔': 0.4, '비장함': 0.4, '신기함/관심': ..."
3,3,1,나는 괴로워했다.,None,서시,윤동주,"절망, 슬픔, 패배/자기혐오","절망, 슬픔, 패배/자기혐오, 죄책감, 힘듦/지침, 비장함","당황/난처, 서러움, 죄책감, 패배/자기혐오","비장함, 슬픔, 패배/자기혐오, 절망, 힘듦/지침","슬픔, 서러움, 절망, 힘듦/지침, 패배/자기혐오","{'패배/자기혐오': 1.0, '슬픔': 0.8, '절망': 0.8, '힘듦/지침'..."
4,4,1,별을 노래하는 마음으로,None,서시,윤동주,"기쁨, 신기함/관심, 즐거움/신남, 흐뭇함(귀여움/예쁨), 뿌듯함","기쁨, 뿌듯함, 감동/감탄, 슬픔, 비장함, 아껴주는, 환영/호의, 기대감","고마움, 기대감, 기쁨, 아껴주는, 흐뭇함(귀여움/예쁨)","감동/감탄, 기대감, 기쁨, 아껴주는, 행복","즐거움/신남, 기대감, 기쁨, 행복","{'기쁨': 1.0, '기대감': 0.8, '아껴주는': 0.6, '감동/감탄': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7617,7617,482,훤조때문에마멸되는몸이다. 모두가소년이라고들그리는데노야인기색이많다. 혹형에씻기워서산반...,None,가외가전,이상,"의심/불신, 불평/불만, 패배/자기혐오, 힘듦/지침, 지긋지긋, 서러움, 공포/무서...","패배/자기혐오, 불쌍함/연민, 공포/무서움, 경악, 불안/걱정, 역겨움/징그러움","깨달음, 불안/걱정, 슬픔, 안타까움/실망, 절망, 패배/자기혐오","경악, 당황/난처, 부담/안_내킴, 힘듦/지침, 절망","힘듦/지침, 불평/불만, 불안/걱정, 깨달음, 절망","{'절망': 0.8, '패배/자기혐오': 0.6, '힘듦/지침': 0.6, '불안/..."
7618,7618,482,반추한다. 노파니까. 맞은편평활한유리위에해소된정체를도포한졸음오는혜택이뜬다. 꿈─꿈─...,None,가외가전,이상,"깨달음, 서러움, 절망, 의심/불신, 패배/자기혐오, 불안/걱정, 지긋지긋, 힘듦/...","깨달음, 재미없음, 절망, 경악, 역겨움/징그러움, 부담/안_내킴, 불쌍함/연민, ...","불안/걱정, 슬픔, 안타까움/실망, 절망","슬픔, 절망, 힘듦/지침","신기함/관심, 힘듦/지침, 깨달음, 절망, 불안/걱정","{'절망': 1.0, '힘듦/지침': 0.6, '깨달음': 0.6, '불안/걱정':..."
7619,7619,482,눈에띄우지않는폭군이잠입하였다는소문이있다. 아기들이번번이애총이되고되고한다. 어디로피해...,None,가외가전,이상,"불안/걱정, 지긋지긋, 증오/혐오, 짜증, 화남/분노, 역겨움/징그러움, 절망, 공...","불쌍함/연민, 공포/무서움, 증오/혐오, 역겨움/징그러움, 경악, 당황/난처, 불안...","불안/걱정, 슬픔, 안타까움/실망, 증오/혐오","경악, 공포/무서움, 절망, 패배/자기혐오, 힘듦/지침","역겨움/징그러움, 증오/혐오, 당황/난처, 공포/무서움, 경악, 절망","{'공포/무서움': 0.8, '증오/혐오': 0.8, '절망': 0.6, '역겨움/..."
7620,7620,483,장난감신부살결에서 이따금 우유내음새가 나기도한다. 머(ㄹ)지아니하여 아기를낳으려나보...,1밤,I WED A TOY BRIDE,이상,"불안/걱정, 힘듦/지침, 아껴주는, 안타까움/실망, 슬픔","기대감, 불안/걱정, 우쭐댐/무시함, 화남/분노, 의심/불신, 힘듦/지침","기대감, 불안/걱정, 아껴주는, 안타까움/실망, 의심/불신","깨달음, 놀람, 불안/걱정, 힘듦/지침","흐뭇함(귀여움/예쁨), 의심/불신, 불평/불만, 짜증","{'불안/걱정': 0.8, '의심/불신': 0.6, '힘듦/지침': 0.6, '기대..."


## 3. Building a Vector Database Using the Processed DataFrame and the Latest KcELECTRA Model
- Backbone model : KcELECTRA v2022
- Vector Store : FAISS (Facebook AI Similarity Search)

In [23]:
# 필터링된 데이터프레임 : min-max스케일링이 적용된 감정라벨, 해당 본문, 작가 정보 호출
df_meta = df_sum[df_sum["scaled_labels"].apply(lambda x: len(x) > 0)].reset_index(drop=True)
texts = df_sum["text"].dropna().astype(str).tolist()
authors = df_sum["poet"].dropna().astype(str).tolist()

In [24]:
# 문장들을 LangChain의 Document 형태로 변환
# 벡터 데이터베이스에 저장하기 위해서는 각 텍스트를 Document 객체로 변환해야 함
# Document 객체는 page_content(본문)와 metadata(부가 정보)로 구성되며,
# 여기서는 page_content에 텍스트만 담고 metadata는 비워둠
# 예: Document(page_content="별 헤는 밤", metadata={"poet": "윤동주"})
documents = [Document(page_content=text) for text in texts]

In [27]:
@dataclass
class Document:
    metadata: dict      # 문서의 부가 정보(감정, 시인 등 메타데이터)를 담는 딕셔너리
    page_content: str   # 실제 본문 텍스트

def add_common_labels_to_documents(documents: List[Document], df_cleaned, column_name1="emotion", column_name2="poet"):
    """
    Document 리스트의 metadata에 감정 라벨과 시인 정보를 추가하는 함수.
    
    매개변수:
        documents (List[Document]): Document 객체 리스트
        df_cleaned (pd.DataFrame): 'scaled_labels', 'poet' 열을 포함하는 정제된 데이터프레임
        column_name1 (str): metadata에 추가할 첫 번째 키 이름 (기본값: 'emotion')
        column_name2 (str): metadata에 추가할 두 번째 키 이름 (기본값: 'poet')
    
    반환값:
        List[Document]: metadata에 감정 라벨과 시인 정보가 추가된 Document 객체 리스트
    """
    for i, doc in enumerate(documents):
        if i < len(df_cleaned):
            # 현재 인덱스가 데이터프레임 범위 안에 있을 경우
            # df_cleaned의 감정 라벨(scaled_labels)과 시인(poet) 값을 metadata에 추가
            doc.metadata[column_name1] = df_cleaned["scaled_labels"].iloc[i]
            doc.metadata[column_name2] = df_cleaned["poet"].iloc[i]
        else:
            # df_cleaned 길이를 초과한 경우, 감정 라벨은 빈 리스트로 저장
            doc.metadata[column_name1] = []
    return documents


In [28]:
# add_common_labels_to_documents 함수 호출
# df_sum 데이터프레임에서 감정 라벨(scaled_labels)과 시인(poet) 정보를 꺼내
# Document 객체들의 metadata에 추가
updated_documents = add_common_labels_to_documents(documents, df_sum)

# 결과 출력
# [Document(metadata={'emotion': {'감동/감탄': 0.2, '슬픔': 0.2, '비장함': 1.0, '뿌듯함': 0.4, '서러움': 0.2}, 'poet': '윤동주'}, page_content='죽는 날까지 하늘을 우러러'),
# Document(metadata={'emotion': {'슬픔': 0.2, '기대감': 0.4, '비장함': 1.0, '뿌듯함': 0.2, '부끄러움': 0.6, '깨달음': 0.2, '서러움': 0.2, '불안/걱정': 0.2}, 'poet': '윤동주'}, page_content='한 점 부끄럼이 없기를,'),
# Document(metadata={'emotion': {'감동/감탄': 0.2, '슬픔': 0.4, '기대감': 0.6, '비장함': 0.4, '편안/쾌적': 0.2, '신기함/관심': 0.4, '당황/난처': 0.2, '서러움': 0.2, '불안/걱정': 0.4}, 'poet': '윤동주'}, page_content='잎새에 이는 바람에도'),
# ...]

In [29]:
# KcELECTRA를 이용해 LangChain에서 사용할 Embeddings 클래스 커스텀 정의
class KcELECTRAEmbeddings(Embeddings):
    def __init__(self, model_name: str = "beomi/KcELECTRA-base", device: str = "cpu"):
        # HuggingFace에서 사전학습된 KcELECTRA 토크나이저와 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.device = device

    def _embed(self, text: str):
        """
        입력 텍스트를 KcELECTRA 임베딩 벡터로 변환하는 내부 메서드
        1) 토큰화 → 텐서 변환
        2) 모델 forward pass (gradient 계산 비활성화)
        3) [CLS] 토큰 벡터만 추출
        """
        inputs = self.tokenizer(
            text,
            return_tensors="pt",  # PyTorch 텐서 반환
            padding=True,
            truncation=True,
            max_length=512       # 최대 길이 제한
        ).to(self.device)

        with torch.no_grad():  # 학습이 아니므로 gradient 계산 비활성화
            outputs = self.model(**inputs)
            # last_hidden_state의 첫 번째 토큰([CLS]) 벡터만 사용
            cls_embedding = outputs.last_hidden_state[:, 0, :]

        # 벡터를 NumPy 배열로 변환해 반환
        return cls_embedding.squeeze().cpu().numpy()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """
        여러 개의 문서를 한꺼번에 임베딩하여 리스트로 반환
        """
        return [self._embed(text).tolist() for text in texts]

    def embed_query(self, text: str) -> list[float]:
        """
        단일 쿼리(질의문)를 임베딩하여 리스트로 반환
        """
        return self._embed(text).tolist()


In [30]:
# 벡터 임베딩 모델 로딩 : KcElectra -> backbone 모델로 사용
embedding_model = KcELECTRAEmbeddings()

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [31]:
# FAISS VectorDB 생성
vectorstore = FAISS.from_documents(updated_documents, embedding_model)

In [32]:
# 로컬에 벡터 DB 저장
vectorstore.save_local("./data/poetry_vectorstore")